In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [168]:
age_grups_incidence = pd.read_csv("./final_dataset/age_group_incidence.csv").drop("category_y", axis=1)

/var/folders/3z/db289jnj3cdd97p1_ybqywr00000gn/T/ipykernel_96585/3245627957.py:1: DtypeWarning: Columns (9,15,16,32,34,35,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  age_grups_incidence = pd.read_csv("./final_dataset/age_group_incidence.csv").drop("category_y", axis=1)


In [169]:
age_groups = ["до 1 р.", "1-4 р.", "5-9 р.", "10-14р", "15-19р", 
              "в т.ч. 15-17р",	"20-24р",	"25-29р", "30-34р",
              "35-39р", "40-44р", "45-49р", "50-54р", "55-59р", 
              "60-64р", "65-69р",	"70-74р",	"75-79р",	"80-84р",	"85 та старші"]

age_group_df = [age_grups_incidence[age_grups_incidence["age_group"] == group] for group in age_groups]

Drop missing values for the time being

In [170]:

age_grups_incidence = age_grups_incidence.dropna()

gdp_numeric = pd.to_numeric(age_grups_incidence["gdp"], errors="coerce")
mask_convertible = gdp_numeric.notna()

age_grups_incidence = age_grups_incidence[mask_convertible]

age_grups_incidence["gdp"] = age_grups_incidence.gdp.astype("float64")

age_grups_incidence["normalized_gdp"] = age_grups_incidence["gdp"]/age_grups_incidence["cpi"]

In [171]:
age_grups_incidence.columns

Index(['Unnamed: 0.1', 'year', 'category_x', 'region', 'age_group',
       'incidence', 'nhospotal', 'nbeds', 'ybeds', 'nill', 'nvillage_ill',
       'bed_days', 'dvisits', 'hvisits', 'ndoctors', 'nnursing', 'nx_ray',
       'nflurography', 'nradiology', 'nradlab', 'nсt', 'ncardiogram',
       'ndiaglab', 'nbacter', 'nbiochem', 'ncyto', 'nimun', 'nphysic',
       'nendoscop', 'nultrasound', 'ndialysis', 'gdp', 'Unnamed: 0',
       'air_pollution', 'polluted_dumps', 'not_cleaned_dumps',
       'dumps_not_cleaned_enough', 'num_clearing_plants', 'cpi', 'population',
       'normalized_gdp'],
      dtype='object')

# Splitting datasets to compare linear and log-linear models

In [172]:
train_idx, test_idx = train_test_split(age_grups_incidence.index, test_size=0.3, random_state=42)


X_train = age_grups_incidence.loc[train_idx]
X_test = age_grups_incidence.loc[test_idx]

# Linear-Linear models

In [173]:
def fit_models(predicted: str, ommited_vars: list[str], df: pd.DataFrame) -> dict[str, sm.OLS]:
    models = {}

    for group_name, group_data in df.groupby("age_group"):
        predictors = group_data.copy().drop(columns=ommited_vars + [predicted], errors="ignore")
        
        predictors = predictors.select_dtypes(include=["number"])
        
        X = predictors.copy()
        y = group_data[predicted]
        
        X = sm.add_constant(X)
        
        best_model = sm.OLS(y, X).fit()

        best_aic = np.inf

        to_drop = None

        while len(X.columns) > 0:
            aic_not_changed = True
                
            for col in X.columns:
                temp_X = X.drop(col, axis = 1)
                temp_model = sm.OLS(y, temp_X).fit()
                if temp_model.aic < best_aic:
                    best_aic = temp_model.aic
                    best_model = temp_model
                    to_drop = col
                    aic_not_changed = False

            if aic_not_changed:
                print(group_name)
                break
            X = X.drop(to_drop, axis = 1)

        models[group_name] = best_model

    return models


In [174]:
models = fit_models("incidence", ["age_group", "year", "gdp", "region", "category_x"], X_train)

1-4 р.
10-14р
15-19р
20-24р
25-29р
30-34р
35-39р
40-44р
45-49р
5-9 р.
50-54р
55-59р
60-64р
65-69р
70-74р
75-79р
80-84р
85 та старші
в т.ч. 15-17р
до 1 р.


# Log-linear models

In [175]:
X_train_log = X_train.copy()
X_train_log = X_train_log[X_train_log["incidence"] > 0]
X_train_log["log_incidence"] = np.log(X_train_log["incidence"])
log_models = fit_models("log_incidence", ["incidence","age_group", "year", "gdp", "region", "category_x"], X_train_log)



1-4 р.
10-14р
15-19р
20-24р
25-29р
30-34р
35-39р
40-44р
45-49р
5-9 р.
50-54р
55-59р
60-64р
65-69р
70-74р
75-79р
80-84р
85 та старші
в т.ч. 15-17р
до 1 р.


In [197]:
X_test = X_test[X_test["incidence"] > 0]

is_linear_better_models = {}
is_log_better_models = {}

for group, group_data in X_test.groupby("age_group"):
    lin_X_test_curr_group = group_data[models[group].model.exog_names[1:]].copy()
    log_X_test_curr_group = group_data[log_models[group].model.exog_names[1:]].copy()

    y_lin_pred = models[group].predict(sm.add_constant(lin_X_test_curr_group))+1
    y_log_pred = log_models[group].predict(sm.add_constant(log_X_test_curr_group))
    log_of_lin_preds = np.log(y_lin_pred)

    lin_X_test_curr_group["log(lin)-log"] = log_of_lin_preds - y_log_pred

    lin_X_test_curr_group = lin_X_test_curr_group["log(lin)-log"].fillna(0)

    is_linear_better_models[group] = sm.OLS(group_data["incidence"], sm.add_constant(lin_X_test_curr_group)).fit()

    exp_of_log_preds = np.exp(y_log_pred)
    log_X_test_curr_group["lin-exp(log)"] = y_lin_pred - exp_of_log_preds

    is_log_better_models[group] = sm.OLS(np.log(group_data["incidence"]+1), sm.add_constant(log_X_test_curr_group)).fit()

/Users/vitya/Documents/econometrics/demand_forecasting/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/vitya/Documents/econometrics/demand_forecasting/.env/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [195]:
p_values_is_linear_better = [int(is_linear_better_models[group].pvalues.get("log(lin)-log")>0.05) for group in age_groups]

for i, group in enumerate(age_groups):
    if p_values_is_linear_better[i] == 0:
        print(f"for group {group} log model may be bettr")

for group 5-9 р. log model may be bettr
for group 15-19р log model may be bettr
for group 20-24р log model may be bettr
for group 25-29р log model may be bettr
for group 30-34р log model may be bettr
for group 35-39р log model may be bettr
for group 40-44р log model may be bettr
for group 45-49р log model may be bettr
for group 50-54р log model may be bettr
for group 55-59р log model may be bettr
for group 60-64р log model may be bettr
for group 65-69р log model may be bettr
for group 70-74р log model may be bettr
for group 75-79р log model may be bettr
for group 80-84р log model may be bettr
for group 85 та старші log model may be bettr


In [203]:

p_values_is_log_better = [int(is_log_better_models[group].pvalues.get("lin-exp(log)")>0.05) for group in age_groups]

for i, group in enumerate(age_groups):
    if p_values_is_log_better[i] == 0:
        print(f"for group {group} lin model may be bettr, pvalue is {is_log_better_models[group].pvalues.get('lin-exp(log)')}")

for group 20-24р lin model may be bettr, pvalue is 0.017489980681450582
for group 25-29р lin model may be bettr, pvalue is 0.024364614747046
for group 40-44р lin model may be bettr, pvalue is 0.026840706680787488
for group 65-69р lin model may be bettr, pvalue is 0.018143841212603016
for group 85 та старші lin model may be bettr, pvalue is 0.003107229831960195
